In [3]:
#pip install librosa

# Loading Libraries

In [2]:
import os
import numpy as np
import librosa
import re
import pandas as pd
import glob



## Maximum Duration and Frequency Range Analysis of Call Annotations

This function, calculate_max_dimensions, was developed to identify the maximum time duration and frequency span of selected animal call annotations across multiple files. It processes a list of annotation files, ensuring they contain essential columns such as start/end time and frequency bounds. The function filters for predefined call types (e.g., "rupe a," "g rupe," "growl b") in a case-insensitive manner, then computes each call’s time and frequency range. It tracks and updates the maximum values observed, storing the associated file and call type for reporting. This analysis helps determine the most demanding spectrogram parameters for visualizing all relevant call types effectively.










In [3]:
def calculate_max_dimensions(annotation_files):
    # Initialize tracking variables
    max_time_info = {'value': -float('inf'), 'file': None, 'call_type': None}
    max_freq_info = {'value': -float('inf'), 'file': None, 'call_type': None}
    
    # Allowed call types (case-insensitive)
    allowed_calls = {
        'rupe a', 'rupe b', 'rupe c', 'type 4 a',
        'guttural rupe', 'g rupe', 'growl b', 'trrot'
    }

    for ann_file in annotation_files:
        print(f"\nProcessing file: {ann_file}")
        try:
            # Load annotation file
            df = pd.read_csv(ann_file, sep='\t')
            
            # Check for required columns
            required_columns = ['Begin Time (s)', 'End Time (s)', 
                               'Low Freq (Hz)', 'High Freq (Hz)', 'Annotation']
            if not all(col in df.columns for col in required_columns):
                print(f"  Missing required columns in {ann_file}. Skipping.")
                continue

            # Calculate deltas
            df['Delta Time (s)'] = df['End Time (s)'] - df['Begin Time (s)']
            df['Delta Freq (Hz)'] = df['High Freq (Hz)'] - df['Low Freq (Hz)']
            
            # Filter to allowed calls (case-insensitive)
            mask = df['Annotation'].str.lower().isin(allowed_calls)
            filtered_df = df[mask]
            
            if filtered_df.empty:
                print(f"  No relevant calls found in {ann_file}")
                continue

            # Update maxima
            for _, row in filtered_df.iterrows():
                # Check time
                if row['Delta Time (s)'] > max_time_info['value']:
                    max_time_info.update({
                        'value': row['Delta Time (s)'],
                        'file': ann_file,
                        'call_type': row['Annotation']
                    })
                
                # Check frequency
                if row['Delta Freq (Hz)'] > max_freq_info['value']:
                    max_freq_info.update({
                        'value': row['Delta Freq (Hz)'],
                        'file': ann_file,
                        'call_type': row['Annotation']
                    })

            # Print current status
            print(f"  Current Max Time: {max_time_info['value']:.2f}s ({max_time_info['call_type']})")
            print(f"  Current Max Freq: {max_freq_info['value']:.2f}Hz ({max_freq_info['call_type']})")

        except Exception as e:
            print(f"  Error processing file: {str(e)}")

    return max_time_info, max_freq_info


In [4]:
annotation_path = '/**/*.selections.txt'
annotation_files = glob.glob(annotation_path, recursive=True)

print(f"Found {len(annotation_files)} annotation files.")

if not annotation_files:
    print("No annotation files found. Check your path!")


Found 78 annotation files.


In [ ]:
if annotation_files:
    max_time, max_freq = calculate_max_dimensions(annotation_files)


Processing file: /Users\ADMIN\Desktop\Freelance\Jasho\---GREY SEAL\audio-ml-project\data\Guttural rupe\5711.211013040024.Table.1.selections.txt
  Current Max Time: 1.05s (G rupe)
  Current Max Freq: 498.15Hz (G rupe)

Processing file: /Users\ADMIN\Desktop\Freelance\Jasho\---GREY SEAL\audio-ml-project\data\Guttural rupe\5711.211013050024.Table.1.selections.txt
  Current Max Time: 1.10s (G rupe)
  Current Max Freq: 498.15Hz (G rupe)

Processing file: /Users\ADMIN\Desktop\Freelance\Jasho\---GREY SEAL\audio-ml-project\data\Guttural rupe\5711.211015090024.Table.1.selections.txt
  Current Max Time: 1.32s (G rupe)
  Current Max Freq: 498.15Hz (G rupe)

Processing file: /Users\ADMIN\Desktop\Freelance\Jasho\---GREY SEAL\audio-ml-project\data\Guttural rupe\5711.211015190024.Table.1.selections.txt
  Current Max Time: 1.32s (G rupe)
  Current Max Freq: 611.70Hz (G rupe)

Processing file: /Users\ADMIN\Desktop\Freelance\Jasho\---GREY SEAL\audio-ml-project\data\Guttural rupe\5711.211015200024.Table.

In [6]:
if annotation_files:
    # Create result string
    result = """Global Maximum Results:
    
Max Delta Time: {time_value:.2f} seconds
- Call Type: {time_call}
- File: {time_file}

Max Delta Frequency: {freq_value:.2f} Hz
- Call Type: {freq_call}
- File: {freq_file}
""".format(
        time_value=max_time['value'],
        time_call=max_time['call_type'],
        time_file=max_time['file'],
        freq_value=max_freq['value'],
        freq_call=max_freq['call_type'],
        freq_file=max_freq['file']
    )

    # Save to file
    with open('max_dimensions_results.txt', 'w') as f:
        f.write(result)
    
    # Print results
    print("\nFinal Results:")
    print(result)
else:
    print("No files to process.")



Final Results:
Global Maximum Results:
    
Max Delta Time: 2.05 seconds
- Call Type: G rupe
- File: /Users\ADMIN\Desktop\Freelance\Jasho\---GREY SEAL\audio-ml-project\data\Guttural rupe\5711.211015200024.Table.1.selections.txt

Max Delta Frequency: 2990.31 Hz
- Call Type: Trrot
- File: /Users\ADMIN\Desktop\Freelance\Jasho\---GREY SEAL\audio-ml-project\data\Rupes A and B\5713.210827200002.Table.1.selections.txt

